# Merging and Mapping CISPolicies and Score of Assets in sitescans

In [ ]:
import sys
import time, datetime
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

In [ ]:
def getAssetsInSite(siteID=""):
    assets, page = [], 0
    print("\nGetting Assets of Your Site")
    while True:
        getAssetsInSiteURL = HOSTNAME + "api/3/sites/" + str(siteID) + "/assets?size=100&page=" + str(page)
        print("Hostname : " + getAssetsInSiteURL)
        assetInPage = requests.get(getAssetsInSiteURL, headers=HEADER, verify=False)
        if assetInPage.status_code == 200:
            assetInPage = assetInPage.json()
            if assetInPage["page"]["totalResources"] == 0:
                print("[Not Found] Don't have assets in your site ID")
                sys.exit()
            assets += [[asset["ip"], asset["id"]] for asset in assetInPage["resources"]]
            if (assetInPage["page"]["totalPages"] == page+1) and (assetInPage["page"]["totalResources"] == len(assets)):
                return assets
            else:
                page += 1
        else:
            print("[Error] Request Connection Error.")
            print(assetInPage.status_code())
            sys.exit()

In [ ]:
siteIDs = ['1234']

In [ ]:
siteID = siteIDs[0]
assets = getAssetsInSite(siteID=siteID)
len(assets), assets

In [ ]:
def extractCISPolicy(data={}):
    return {
        "surrogateId": str(data["surrogateId"]),
        "policyID":data["id"],
        "policyName":data["policyName"],
        "ruleCompliance":data["ruleCompliance"],
        "policyTitle":data['title'],
        "benchmarkName":data["benchmarkName"],
        "failedRulesCount": data["failedRulesCount"],
        "passedRulesCount": data["passedRulesCount"]
    }

In [ ]:
def getCISPoliciesofAsset(assetID=None):
    getCISPoliciesofAssetURL = HOSTNAME + 'api/3/assets/' + \
        str(assetID) + '/policies?applicableOnly=true&page=0&size=500'
    print("Hostname : " + getCISPoliciesofAssetURL)
    CISPoliciesofAsset = requests.get(getCISPoliciesofAssetURL,
                                      headers=HEADER, verify=False)
    CISPoliciesofAsset = CISPoliciesofAsset.json()
    if CISPoliciesofAsset["page"]["totalPages"] == 1:
        print("Getting Successful.")
        return [extractCISPolicy(data=b) for b in CISPoliciesofAsset["resources"]]
    else:
        print("[Error] Not Found CIS Policie.")

In [ ]:
def getCISPoliciesofAssets(assets=[]):
    assetsPolicies, policies = [], []
    for asset in assets:
        time.sleep(2)
        policy = getCISPoliciesofAsset(assetID=asset[1])
        assetsPolicies.append({
            "ip": asset[0],
            "assetid": asset[1],
            "policies": policy
        })
        if policy:
            policies += [p["policyTitle"] for p in policy]
    return assetsPolicies, policies

In [ ]:
def findPolicyInAsset(data=[], policy=""):
    for p in data:
        if p["policyTitle"].lower() == policy.lower():
            return str(p["ruleCompliance"])
    return str(0)

In [ ]:
def extractRowfromData(data={}, all_policies=[]):
    data_extract = []
    if data['policies'] != None:
        for a_pol in all_policies:
            data_extract.append(findPolicyInAsset(data=data['policies'], policy=a_pol))
    else:
        data_extract += [str(0) for p in all_policies]
    return data_extract


In [ ]:
def generatePandasData(assetsPolicies=[], policies=[]):
    extractRowfromDatas = []
    all_policies = list(set(policies))
    for ap in assetsPolicies:
        data = [ap["ip"]]
        data += extractRowfromData(data=ap, all_policies=all_policies)
        extractRowfromDatas.append(data)
    return ["Asset"] + all_policies, extractRowfromDatas

In [ ]:
assetsPolicies, policies = getCISPoliciesofAssets(assets=assets)
header, data_rows = generatePandasData(assetsPolicies=assetsPolicies, policies=policies)

In [ ]:
df = pd.DataFrame(data_rows, columns=header)
df

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
df.to_excel ("outputs/" + nowTime + 'policiesMap_' + siteID + '.xlsx', index = False, header=True)